In [26]:
#
# Import Libraries
#

import os
import sys
import pandas as pd
import numpy as np

from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

utilsPath = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\utils'
if utilsPath not in sys.path:
    sys.path.append(utilsPath)

from BasicStats import Xstats, anova_calc, kruskal_calc, sign_calc

In [3]:
#
# Set constants
#

Qworking_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\ALDH4"
Mworking_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4"

# X
xq_path = os.path.join(Qworking_path, "WorkingFiles", "Xq_minus_X_norm.tsv")
mq_path = os.path.join(Mworking_path, "WorkingFiles", "Xm_norm.tsv")

# Feature info
q2i_path = os.path.join(Qworking_path, "WorkingFiles", "q2info.tsv")
m2i_path = os.path.join(Mworking_path, "WorkingFiles", "f2info_TP.tsv")

# metadata
mdata_path = os.path.join(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\ALDH4\WorkingFiles\main_metadata.tsv")

In [4]:
#
# Read data
#

xq = pd.read_csv(xq_path, sep='\t', index_col=0)
xm = pd.read_csv(mq_path, sep='\t', index_col=0)

#q2i = pd.read_csv(q2i_path, sep='\t')
#m2i = pd.read_csv(m2i_path, sep='\t')

mdata = pd.read_csv(mdata_path, sep='\t').set_index('Seqn')

In [32]:
from scikit_posthocs import posthoc_tukey_hsd

In [42]:
[pairwise_tukeyhsd(x,y).pvalues for i in range(20)]

[array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615]),
 array([0.54937335, 0.20306887, 0.04015615])]

In [48]:
g2s = {i:j.index.tolist() for i,j in mdata.groupby('Group')}

# qq

g2sq = {
    i: [k for k in j if k in xq.index] for i,j in g2s.items()
}

myanova = xq.join(mdata['Group'], how='left').groupby('Group').agg(list).to_dict()
res_anova = {}

for key,obj in myanova.items():
    res_anova[key] = {}
    x,y = list(zip(*[(v, g) for g in ['PBS', 'B1-8', 'A12'] for v in obj[g]]))
    res_anova[key]['anova'] = f_oneway(*list(obj.values())).pvalue
    tukey = pairwise_tukeyhsd(x,y) 
    res_anova[key]['anova_ph'] = tukey.pvalues if res_anova[key]['anova']<0.1 else None

In [76]:
from itertools import combinations
gg = list(combinations(tukey.groupsunique, 2))


qstat = pd.DataFrame({
    i: {'anova': j['anova']}
    for i,j in res_anova.items()
}).T.join(
pd.DataFrame({
    i: {'_'.join(k):j['anova_ph'][n] if type(j['anova_ph'])!=type(None) else None for n,k in enumerate(gg)}
    for i,j in res_anova.items()
}).T
)

In [82]:
qstat['LFC_A12_B1-8'] = xq.loc[g2sq['A12']].mean()-xq.loc[g2sq['B1-8']].mean()
qstat['LFC_A12_PBS'] = xq.loc[g2sq['A12']].mean()-xq.loc[g2sq['PBS']].mean()

In [83]:
g2s = {i:j.index.tolist() for i,j in mdata.groupby('Group')}

# qq

g2sm = {
    i: [k for k in j if k in xm.index] for i,j in g2s.items()
}

myanova = xm.join(mdata['Group'], how='left').groupby('Group').agg(list).to_dict()
res_anova = {}

for key,obj in myanova.items():
    res_anova[key] = {}
    x,y = list(zip(*[(v, g) for g in ['PBS', 'B1-8', 'A12'] for v in obj[g]]))
    res_anova[key]['anova'] = f_oneway(*list(obj.values())).pvalue
    tukey = pairwise_tukeyhsd(x,y) 
    res_anova[key]['anova_ph'] = tukey.pvalues if res_anova[key]['anova']<0.1 else None
from itertools import combinations
gg = list(combinations(tukey.groupsunique, 2))

In [ ]:
mstat = pd.DataFrame({
    i: {'anova': j['anova']}
    for i,j in res_anova.items()
}).T.join(
pd.DataFrame({
    i: {'_'.join(k):j['anova_ph'][n] if type(j['anova_ph'])!=type(None) else None for n,k in enumerate(gg)}
    for i,j in res_anova.items()
}).T
)
mstat['LFC_A12_B1-8'] = xm.loc[g2sm['A12']].mean()-xm.loc[g2sm['B1-8']].mean()
mstat['LFC_A12_PBS'] = xm.loc[g2sm['A12']].mean()-xm.loc[g2sm['PBS']].mean()

In [8]:
qstat.to_csv('Xq_stats.tsv', sep='\t')
mstat.to_csv('Xm_stats.tsv', sep='\t')